![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/PUBLIC_HEALTH_CLASSIFIER_DL.ipynb)

# **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

# **Install dependencies**

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

# **Import dependencies into Python and start the Spark session**

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


# **MODELS**

## **classifierdl_vaccine_sentiment**

In [4]:
def run_pipeline(model, text, lang = 'en'):  
  document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("sentence")

  tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

  bert_embeddings = BertEmbeddings.pretrained("bert_embeddings_phs_bert", "en", "public/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\

  embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["sentence", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

  classifierdl = ClassifierDLModel.pretrained(model, "en", "clinical/models")\
    .setInputCols(['sentence_embeddings'])\
    .setOutputCol('class')

  pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        bert_embeddings,
        embeddingsSentence,
        classifierdl
    ])

  df = spark.createDataFrame(text, StringType()).toDF("text")
  results = pipeline.fit(df).transform(df)
   
  print("\n")
  print("<----------------- MODEL NAME:","\033[1m" + model + "\033[0m"," ----------------- >")
  
  res = results.select( F.explode(F.arrays_zip("sentence.result", 
                                               "class.result",
                                               "class.metadata")).alias("col"))\
               .select( F.expr("col['1']").alias("prediction"),
                        F.expr("col['2']").alias("confidence"),
                        F.expr("col['0']").alias("sentence"))
  if res.count()>1:
    udf_func = F.udf(lambda x,y:  x[str(y)])
    print("\n",model,"\n") 
    res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=False)


In [5]:
model = "classifierdl_vaccine_sentiment"

In [6]:
sample_texts = [""" Perhaps when there s a  COVID 19 vaccine in the coming months  or years , it can help countries like   Pakistan increase national immunization stats if routine immunization is coupled with the virus jab drops.   Should the state, however, make vaccination a mandatory citizenship duty. """,
""" Today  I received my second dose of the  COVID 19 vaccine. When it becomes available to you, don t wait   get vaccinated.  It s safe, easy, and it saves lives. """,
""" I got my mom scheduled for the   Covid 19 vaccine.  A great relief to me, to be honest. """,
""" It feels really exciting to have a personal connection to the province s vaccine numbers.   My step dad s 92 year old mother got her first dose yesterday.""",
""" The current oxford vaccine is based off the work they did on the non mild coronavirus forms of  SARS and  MERS.   But since they were contained, the urgency to continue the work was reduced until  COVID 19. """,
""" Got the covid vaccine tonight.... so far side effects for me. Super weak  exhausted. Injection site and arm hurts  AF Feel like  I smoked a fat. Other than that... feeling like a million bucks for doing my part. """]

In [7]:
run_pipeline(model, sample_texts)

bert_embeddings_phs_bert download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
classifierdl_vaccine_sentiment download started this may take some time.
Approximate size to download 23.1 MB
[OK!]


<----------------- MODEL NAME: classifierdl_vaccine_sentiment  ----------------- >

 classifierdl_vaccine_sentiment 

+----------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|confidence|sentence                                                                                                                                                                                                                                                                                       |
+----------+----------+--------

## **classifierdl_health_mentions**

In [8]:
model = "classifierdl_health_mentions"

In [9]:
sample_texts = [
"""Got sumatriptan for this week long migraine  I thought these were over as a kid  but I guess not  Thought I could just baby it a couple days with water and pedialyte and excedrin  Nope    Why am I so stubborn about continuing to  DO THINGA when I m obviously not well""",
"""That Thunder trade alert just now from  almost gave me a heart attack  Was bracing myself to see THUNDER SENDS RUSS TO Wow That was the briefest emotional roller coaster ever""",
"""The sickle cell clinic of  still goes on  It is even more for the next 2 months  Ever heard of childhood stroke  One of the leading causes of childhood stroke is sickle cell  Amidst cancer and other""",
"""In 2015 I suffered a stroke  This prevented me from playing Rugby League again  It was difficult physically and mentally to recover from  bit four years later  here I am  about to embark on this   Read my story and please donate if you can""",
"""i respect the fuck out of the lesbian who was in the bar bathroom  one holer  directly before me and dropped a gnarly rank dookie like make a raccoon cough type shit""",  
"""Many of us have witnessed the sad  steady march of Alzheimer s disease as it destroys memory and thinking ability  But what about the physical effects of Alzheimer s which also are significant  but which we tend not to think about as much"""
]

In [10]:
run_pipeline(model, sample_texts)

bert_embeddings_phs_bert download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
classifierdl_health_mentions download started this may take some time.
Approximate size to download 22.9 MB
[OK!]


<----------------- MODEL NAME: classifierdl_health_mentions  ----------------- >

 classifierdl_health_mentions 

+------------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction        |confidence|sentence                                                                                                                                                                                                                                                                   |
+------------------+----------+------------------------------